In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
import xlrd

In [3]:
df_case = pd.read_csv("../../../02_Data//01_COVID/covid_us_county.csv")
df_case.dropna(inplace=True)
df_case = df_case[['fips','lat','long','date','cases','deaths']]
df_case['fips'] = df_case.fips.astype(int)

In [4]:
def Seven_Day_Moving_Average(df,minimum_case=5):
    df_c = df.copy()
    fips = list(set(df_c['fips']))
    
    mData = []
    for fip in tqdm(fips):
        df = df_c[df_c['fips']==fip]
        df = df[df['cases']>=minimum_case]
        df = df.reset_index(drop=True)
        df = df.reset_index()
        
        df = df.to_dict('records')
        # 7 day moving average
        for i in range(len(df)-6):
            mdf = df[i:i+7]
            mdf_case = [m['cases'] for m in mdf]
            mdf_death = [m['deaths'] for m in mdf]
            df[i+3]['cases_7'] = np.sum(mdf_case)/7
            df[i+3]['deaths_7'] = np.sum(mdf_death)/7
            df[i+3]['day_index'] = df[i+3]['index']
            mData.append(df[i+3])
    
    # Log growth rate
    for i in range(len(mData)-1):
        mData[i+1]['delta_case'] = np.log(mData[i+1]['cases_7']) - np.log(mData[i]['cases_7'])
        pass
    del mData[0]
    return pd.DataFrame(mData)
    

In [5]:
df_case_delta = Seven_Day_Moving_Average(df_case)

100%|█████████████████████████████████████████████████████████████████████████████| 3241/3241 [00:24<00:00, 133.25it/s]


In [8]:
df_case_delta['date'] = pd.to_datetime(df_case_delta['date'])
df_case_delta['month'] = df_case_delta['date'].dt.month
print(df_case_delta)

        index   fips        lat        long       date  cases  deaths  \
0           4  41001  44.709156 -117.674988 2020-07-08      8       0   
1           5  41001  44.709156 -117.674988 2020-07-09      8       0   
2           6  41001  44.709156 -117.674988 2020-07-10      9       0   
3           7  41001  44.709156 -117.674988 2020-07-11     11       0   
4           8  41001  44.709156 -117.674988 2020-07-12     12       0   
...       ...    ...        ...         ...        ...    ...     ...   
577041    178  39069  41.333964  -84.068306 2020-10-23    488      17   
577042    179  39069  41.333964  -84.068306 2020-10-24    498      17   
577043    180  39069  41.333964  -84.068306 2020-10-25    503      17   
577044    181  39069  41.333964  -84.068306 2020-10-26    531      17   
577045    182  39069  41.333964  -84.068306 2020-10-27    535      17   

           cases_7  deaths_7  day_index  delta_case  month  
0         7.285714       0.0          4    0.125163      7  
1